In [1]:
%load_ext autoreload
%autoreload 2.0
%matplotlib inline

In [2]:
import os
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import numpy as np
import sys
sys.path.insert(0, '../.')
from data.detection_dataset_loader import *
from scipy.spatial.transform import Rotation as R
from data.data_utils.data_reader import *
from data.data_utils.reader_utils import *
import math
import numpy.matlib as npm

from scipy.interpolate import RegularGridInterpolator, NearestNDInterpolator

/Users/apple/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
def get_augmentation_parameters( training):
        if training:

                    if np.random.random_sample() >= 0.5:
                        image_translate_x = random.randint(-50, 50)
                    else:
                        image_translate_x = 0
                    if np.random.random_sample() >= 0.5:
                        image_translate_y = random.randint(-25, 25)
                    else:
                        image_translate_y = 0

                    if np.random.random_sample() >= 0.5:
                        translate_x = random.randint(-5, 5)
                    else:
                        translate_x = 0
                    if np.random.random_sample() >= 0.5:
                        translate_y = random.randint(-5, 5)
                    else:
                        translate_y = 0

                    if np.random.random_sample() >= 0.8:
                        translate_z = random.random() - 0.5
                    else:
                        translate_z = 0

                    if np.random.random_sample() >= 0.3:
                        ang = random.randint(-5, 5)
                    else:
                        ang = 0

                    r = R.from_rotvec(np.radians(ang) * np.array([0, 0, 1]))
                    rot = r.as_dcm()
                    rot = np.append(rot, np.array([[0,0,0]]), axis=0)
                    rot = np.append(rot, np.array([[0],[0],[0],[1]]), axis=1)

                    tr_x = translate_x
                    tr_y = translate_y
                    tr_z = translate_z
                    tr = np.array([[tr_x], [tr_y], [tr_z], [0]])
                    
                    translate_x = 0
                    translate_y = 0
                    translate_z = 0
                    
                    sc_x = 1
                    sc_y = 2
                    sc_z = 1

#                     if np.random.random_sample() >= 0.5:
#                        sc_x += ((random.random() * 2) - 1.) / 10.

#                     if np.random.random_sample() >= 0.5:
#                        sc_y += ((random.random() * 2) - 1.) / 10.


                    sc = np.array([[sc_x, 0, 0, 0], [0, sc_y, 0, 0], [0, 0, sc_z, 0], [0, 0, 0, 1]])
                    
                    fliplr = np.random.random_sample() >= 0.5

        else:
                    image_translate_x = 0
                    image_translate_y = 0

                    translate_x = 0
                    translate_y = 0
                    translate_z = 0
                    ang = 0

                    r = R.from_rotvec(np.radians(0) * np.array([0, 0, 1]))
                    rot = r.as_dcm()
                    rot = np.append(rot, np.array([[0,0,0]]), axis=0)
                    rot = np.append(rot, np.array([[0],[0],[0],[1]]), axis=1)

                    tr_x = 0
                    tr_y = 0
                    tr_z = 0
                    tr = np.array([[tr_x], [tr_y], [tr_z], [0]])

                    sc_x = 1
                    sc_y = 1
                    sc_z = 1
                    sc = np.array([[sc_x, 0, 0, 0], [0, sc_y, 0, 0], [0, 0, sc_z, 0], [0, 0, 0, 1]])
                    
                    fliplr = False

        return rot, tr, sc, image_translate_x, image_translate_y, ang, fliplr

In [4]:
training=True
base_path = '/Users/apple/Desktop/Master2/data/'
list_files = list(map(lambda x: x.split('.')[0], os.listdir(base_path+'/training2/label_2/')))
# random.seed(0)
# random.shuffle(list_files)

# camera_paths = list(map(lambda x: base_path+'/data_object_image_3/training/image_3/' + x + '.png', list_files))
# lidar_paths = list(map(lambda x: base_path+'/data_object_velodyne/training/velodyne/' + x + '.bin', list_files))
label_paths = list(map(lambda x: base_path + '/training2/label_2/' + x + '.txt', list_files))
calib_paths = list(map(lambda x: base_path + '/data_object_calib/training/calib/' + x + '.txt', list_files))
        
ln = int(len(list_files) * 0.5)
final_sample = len(list_files)

        
list_files = list_files[:len(list_files)]
list_label_paths = label_paths[:len(list_files)]
list_calib_paths = calib_paths[:len(list_files)]

In [5]:
def get_target2(labels, directions, anchors=np.array([3.9, 1.6, 1.5]), input_size=(512, 448), output_size=(128, 112)):
    # ASSUMPTION: I will assume that the anchors contain a record for the height
    """
    - calculate the ratio = input size / final output size
    - divide each of the x, y and z by the ratio
    - subtract each of the x, y and z ....
    """

    ratio = input_size[0] // output_size[0]
    y_target = np.zeros((output_size[0], output_size[1], 2, 9), np.float32)
    old_angles = []
    angles = []
    angles_0 = []
    angles_1 = []
    for i in range(len(labels)):
        label_i = np.array(labels[i])

        x = int(label_i[0]/ratio)
        y = int(label_i[1]/ratio)

        if x >= output_size[0]:
            x = output_size[0] - 1
        if y >= output_size[1]:
            y = output_size[1] - 1

        if x < 0 or y < 0:
            continue

        label_i[0:2] = label_i[0:2] / (ratio*1.0)
        label_i[2] = label_i[2] / 40.
        old_angles.append(label_i[6])
        temp = label_i[6]
#         if temp < 0:
#             temp += np.pi

#         angle = temp * 57.2958
#         if (angle >= 0 and angle <= 45) or (angle >= 135 and angle <= 225) or (angle >= 315 and angle <= 360):
#             k = 0
#         else:
#             k = 1

        angle = temp * 57.2958
        if (angle >= 0 and angle <= 45) or (angle <= 0 and angle >= -45)\
            or (angle >= 135 and angle <= 180) or (angle <= -135 and angle >= -180):
            if angle >= 135:
                label_i[6] = label_i[6] - np.pi
            elif angle <= -135:
                label_i[6] = label_i[6] + np.pi
            k = 0
            angles_0.append(label_i[6])
        else:
            if angle < 0:
                label_i[6] = label_i[6] + np.pi
#             elif angle <= -90:
#                 label_i[6] = label_i[6] + np.pi
            angles_1.append(label_i[6])
            k = 1

#         if label_i[6] >= 3 * np.pi / 4:
#             label_i[6] = label_i[6] - np.pi
        
        label_i[6] = label_i[6] - k * (np.pi/2)
        
        angles.append(label_i[6])

        anchor = np.array([x+0.5, y+0.5, 1., anchors[0], anchors[1], anchors[2]])
    
        label_i[:3] = (label_i[:3] - anchor[:3]) / anchor[3:6]
        label_i[3:6] = np.log(label_i[3:6]/anchors)

        y_target[x, y, k, :7] = label_i
        y_target[x, y, k, 7:8] = [directions[i]]
        y_target[x, y, k, 8:9] = [1]
        
    return old_angles, angles, angles_0, angles_1
#     return y_target

In [6]:
list_angles = []
list_original_angles = []
list_angles_0 = []
list_angles_1 = []
for i_c in range(len(list_files)):
    rot, tr, sc, image_translate_x, image_translate_y, ang, fliplr = get_augmentation_parameters(False)
    data_reader_obj = DataReader(None,
                                 list_calib_paths[i_c], 
                                 list_label_paths[i_c], 
                                 None, 
                                 rot, sc, tr, ang, image_translate_x, image_translate_y, get_actual_dims=True, fliplr=fliplr)
    _, label, directions = data_reader_obj.label_reader.read_label()
    old_angles, angles, angles_0, angles_1 = get_target2(label, directions,  anchors=np.array([3.9, 1.6, 1.5]))
    list_angles.extend(angles)
    list_angles_0.extend(angles_0)
    list_angles_1.extend(angles_1)
    list_original_angles.extend(old_angles)


In [7]:
list_angles = np.array(list_angles)
list_original_angles = np.array(list_original_angles)
list_angles_0 = np.array(list_angles_0)
list_angles_1 = np.array(list_angles_1)

In [8]:
np.max(list_angles), np.min(list_angles)

(0.7815926535897932, -0.7815926535897932)

In [9]:
np.max(list_angles_0), np.min(list_angles_0)

(0.7815926535897932, -0.7815926535897932)

In [10]:
np.max(list_angles_1), np.min(list_angles_1)

(2.351592653589793, 0.79)

In [11]:
np.max(list_original_angles), np.min(list_original_angles)

(3.14, 0.0)

In [19]:
np.max(list_angles), np.min(list_angles)

(0.7800000000000002, -0.7815926535897932)

In [20]:
np.max(list_original_angles), np.min(list_original_angles)

(3.14, 0.0)

In [23]:
np.pi, np.pi/2, np.pi/4

(3.141592653589793, 1.5707963267948966, 0.7853981633974483)